In [2]:
import os
import datasets
import numpy as np
import re
import nltk
from gensim.models.keyedvectors import KeyedVectors

nltk.download('stopwords')
from nltk.corpus import stopwords

c:\users\keshavpc\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keshavPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
dataset_path = os.path.join("data", "cornell")
data = datasets.readCornellData(dataset_path, max_len=100000)
data[:5]

100%|█████████████████████████████████████████████████████████████████████████| 83097/83097 [00:03<00:00, 21669.50it/s]


[('can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again',
  'well i thought wed start with pronunciation if thats okay with you'),
 ('well i thought wed start with pronunciation if thats okay with you',
  'not the hacking and gagging and spitting part please'),
 ('not the hacking and gagging and spitting part please',
  'okay then how bout we try out some french cuisine saturday night'),
 ('youre asking me out thats so cute whats your name again', 'forget it'),
 ('no no its my fault we didnt have a proper introduction', 'cameron')]

In [14]:
data

[('can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again',
  'well i thought wed start with pronunciation if thats okay with you'),
 ('well i thought wed start with pronunciation if thats okay with you',
  'not the hacking and gagging and spitting part please'),
 ('not the hacking and gagging and spitting part please',
  'okay then how bout we try out some french cuisine saturday night'),
 ('youre asking me out thats so cute whats your name again', 'forget it'),
 ('no no its my fault we didnt have a proper introduction', 'cameron'),
 ('cameron',
  'the thing is cameron im at the mercy of a particularly hideous breed of loser my sister i cant date until she does'),
 ('the thing is cameron im at the mercy of a particularly hideous breed of loser my sister i cant date until she does',
  'seems like she could get a date easy enough'),
 ('why',
  'unsolved mystery she used to be really popular when she started high sc

In [11]:
dataset_path = os.path.join("data", "opensubs/OpenSubtitles")
data = datasets.readOpensubsData(dataset_path, max_len=100000)

Loading OpenSubtitles conversations in data\opensubs/OpenSubtitles.


OpenSubtitles data files: 0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [15]:
data = [[sentences[0].split(), sentences[1].split()]for sentences in data]
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
data = [sentences for sentences in data if len(sentences[0]) >= 3]
data = [sentences for sentences in data if len(sentences[0]) <= 20]
data = [sentences for sentences in data if len(sentences[1]) >= 3]
data = [sentences for sentences in data if len(sentences[1]) <= 20]

In [17]:
data

[[['well',
   'i',
   'thought',
   'wed',
   'start',
   'with',
   'pronunciation',
   'if',
   'thats',
   'okay',
   'with',
   'you'],
  ['not',
   'the',
   'hacking',
   'and',
   'gagging',
   'and',
   'spitting',
   'part',
   'please']],
 [['not',
   'the',
   'hacking',
   'and',
   'gagging',
   'and',
   'spitting',
   'part',
   'please'],
  ['okay',
   'then',
   'how',
   'bout',
   'we',
   'try',
   'out',
   'some',
   'french',
   'cuisine',
   'saturday',
   'night']],
 [['gosh', 'if', 'only', 'we', 'could', 'find', 'kat', 'a', 'boyfriend'],
  ['let', 'me', 'see', 'what', 'i', 'can', 'do']],
 [['cesc', 'ma', 'tete', 'this', 'is', 'my', 'head'],
  ['right', 'see', 'youre', 'ready', 'for', 'the', 'quiz']],
 [['how',
   'is',
   'our',
   'little',
   'find',
   'the',
   'wench',
   'a',
   'date',
   'plan',
   'progressing'],
  ['well', 'theres', 'someone', 'i', 'think', 'might', 'be']],
 [['how', 'do', 'you', 'get', 'your', 'hair', 'to', 'look', 'like', 'that'],


In [21]:
wv_embeddings = KeyedVectors.load_word2vec_format(
    'data/GoogleNews-vectors-negative300.bin.gz',
    binary=True
)

In [9]:
from collections import defaultdict
word_count = defaultdict(int)
for sentences in data:
    for word in sentences[0]:
        word_count[word] += 1
    for word in sentences[1]:
        word_count[word] += 1

In [22]:
wc_threshold = 50

In [23]:
# remove the lines with unknown words
data_filtered = []
for sentence1, sentence2 in data:
    valid = True
    for word in sentence1 + sentence2:
        if word not in wv_embeddings or word_count[word] < wc_threshold:
            valid = False
            break
    if valid:
        data_filtered.append([sentence1, sentence2])

NameError: name 'word_count' is not defined

In [12]:
len(data_filtered)

311236

In [13]:
data = data_filtered

In [14]:
data.sort(key=lambda x:(len(x[0]), len(x[1])))

In [15]:
data[9]

[['thanks', 'very', 'much'], ['remember', 'your', 'bag']]

In [16]:
word_set = set()
for sentence1, sentence2 in data:
    word_set |= set(sentence1)
    word_set |= set(sentence2)

In [17]:
start_token = "[START]"
end_token = "[END]"
pad_token = "[PAD]"

In [18]:
idx_to_word = [start_token, end_token, pad_token] + [None] * len(word_set)
word_to_idx = {start_token:0, end_token:1, pad_token:2}
index = 3
for word in word_set:
    word_to_idx[word] = index
    idx_to_word[index] = word
    index += 1

In [19]:
start_idx, end_idx = 0, 1

In [20]:
embedding_dim = 300
word_embeddings = np.zeros((len(word_set)+3, embedding_dim), dtype='float32')
word_embeddings[0,:] = 0.
word_embeddings[1,:] = 1.
word_embeddings[2,:] = -1.
for i in range(3, len(word_set)+3):
    word_embeddings[i, :] = wv_embeddings[idx_to_word[i]]

In [21]:
for sentences in data:
    sentences[1].append(end_token)

In [22]:
input_lengths = [len(sentences[0]) for sentences in data]
ground_truth_lengths = [len(sentences[1]) for sentences in data]

In [23]:
# add padding
max_input_lengths = max(input_lengths)
max_ground_truth_lengths = max(ground_truth_lengths)
input_sentences = []
ground_truth_sentences = []
for sentences in data:
    input_sentences.append(sentences[0] + [pad_token]*(max_input_lengths-len(sentences[0])))
    ground_truth_sentences.append(sentences[1] + [pad_token]*(max_ground_truth_lengths-len(sentences[1])))

In [24]:
input_sentences_idx = [[word_to_idx[word] for word in sentence] for sentence in input_sentences]
ground_truth_sentences_idx = [[word_to_idx[word] for word in sentence] for sentence in ground_truth_sentences]

In [25]:
input_sentences_idx = np.array(input_sentences_idx)

In [26]:
ground_truth_sentences_idx = np.array(ground_truth_sentences_idx)
input_lengths = np.array(input_lengths)
ground_truth_lengths = np.array(ground_truth_lengths)

In [27]:
def batch_generator(batch_size,
                    input_sentences_idx,
                    ground_truth_sentences_idx,
                    input_lengths,
                    ground_truth_lengths):
    index = 0
    while index < len(input_sentences_idx):
        batch_input_length = input_lengths[index:index+batch_size]
        batch_input_sentences_idx = input_sentences_idx[index:index+batch_size, :batch_input_length.max()]
        batch_ground_truth_length = ground_truth_lengths[index:index+batch_size]
        batch_ground_truth_sentences_idx = ground_truth_sentences_idx[index:index+batch_size, :batch_ground_truth_length.max()]
        yield (batch_input_sentences_idx, batch_input_length,
              batch_ground_truth_sentences_idx, batch_ground_truth_length)
        index += batch_size

In [65]:
vocab_size = len(word_set) + 3
num_units = 512
embedding_size = 300
num_encoder_layers = 1
num_decoder_layers = 1

In [29]:
import tensorflow as tf
from tensorflow.contrib import layers

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
def text_prepare(text):
    """Performs tokenization and simple preprocessing."""
    
    replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
    good_symbols_re = re.compile('[^0-9a-z #+_]')

    text = text.lower()
    text = replace_by_space_re.sub(' ', text)
    text = good_symbols_re.sub('', text)

    return text.strip()

In [66]:
class ChatBot:
    def __init__(self):
        self.declare_placeholders()
        self.build_input_encoder()
        self.build_decoder()
        self.define_loss_and_train()
    
    def declare_placeholders(self):
        """Specifies placeholders for the model."""
        # Placeholders for input and its actual lengths.
        self.input_batch = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input_batch')
        self.input_batch_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_batch_lengths')

        # Placeholders for groundtruth and its actual lengths.
        self.ground_truth = tf.placeholder(shape=(None, None), dtype=tf.int32, name='ground_truth')
        self.ground_truth_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='ground_truth_lengths')

        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
        self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

    def build_input_encoder(self):
        with tf.variable_scope('input_encoder') as input_encoder_scope:
#             random_initializer = tf.random_uniform((vocab_size, embedding_size), -1.0, 1.0)
#             self.embeddings = tf.Variable(initial_value=random_initializer, name='embeddings', dtype=tf.float32) 
            self.embeddings = tf.Variable(initial_value=word_embeddings, name='embeddings', dtype=tf.float32) 

            # Perform embeddings lookup for self.input_batch. 
            self.input_batch_embedded = tf.nn.embedding_lookup(self.embeddings, self.input_batch)
            # Create encoder cells
            rnn_layers = []
            for i in range(num_encoder_layers-1):
                with tf.variable_scope('input_encoder_rnn_layer' + str(i + 1)) as scope:
                    cell = tf.nn.rnn_cell.GRUCell(num_units)
                    cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                    rnn_layers.append(cell)
            with tf.variable_scope('input_encoder_rnn_layer' + str(num_encoder_layers)) as scope:
                cell = tf.nn.rnn_cell.GRUCell(num_units)
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                rnn_layers.append(cell)
            encoder_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
            self.input_encoder_outputs, self.final_input_encoder_state = tf.nn.dynamic_rnn(
                encoder_cell,
                self.input_batch_embedded,
                sequence_length=self.input_batch_lengths,
                dtype=tf.float32
            )
            self.final_input_encoder_state = self.final_input_encoder_state[-1]

    def build_decoder(self):
        batch_size = tf.shape(self.input_batch)[0]
        start_tokens = tf.fill([batch_size], start_idx)
        ground_truth_as_input = tf.concat([tf.expand_dims(start_tokens, 1), self.ground_truth], 1)
        self.ground_truth_embedded = tf.nn.embedding_lookup(
            self.embeddings, ground_truth_as_input)
        train_helper = tf.contrib.seq2seq.TrainingHelper(self.ground_truth_embedded,
                                                         self.ground_truth_lengths)
        infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(self.embeddings, start_tokens, end_idx)

        def decode(helper, scope, reuse=None):
            """Creates decoder and return the results of the decoding with a given helper."""
            with tf.variable_scope(scope, reuse=reuse):
                # Create GRUCell with dropout. Do not forget to set the reuse flag properly.
                rnn_layers = []
                for i in range(num_decoder_layers-1):
                    with tf.variable_scope('decoder_rnn_layer' + str(i + 1)) as scope:
                        decoder_cell = tf.contrib.rnn.GRUCell(num_units=num_units, reuse=reuse)
                        decoder_cell = tf.contrib.rnn.DropoutWrapper(decoder_cell, input_keep_prob=self.dropout_ph)
                        rnn_layers.append(decoder_cell)
                with tf.variable_scope('decoder_rnn_layer' + str(num_decoder_layers)) as scope:
                    decoder_cell = tf.contrib.rnn.GRUCell(num_units=num_units, reuse=reuse)
                    decoder_cell = tf.contrib.rnn.DropoutWrapper(decoder_cell, input_keep_prob=self.dropout_ph)
                    rnn_layers.append(decoder_cell)
                decoder_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
                # Create attention
#                 attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
#                     num_units=num_units, memory=tf.split(self.input_encoder_outputs, num_or_size_splits=2, axis=-1)[0],
#                     memory_sequence_length=self.input_batch_lengths)
#                 decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
#                     decoder_cell, attention_mechanism, attention_layer_size=num_units)
                # Create a projection wrapper.
                decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(decoder_cell, vocab_size, reuse=reuse)
                # Create BasicDecoder, pass the defined cell, a helper, and initial state.
                # The initial state should be equal to the final state of the encoder!
                decoder = tf.contrib.seq2seq.BasicDecoder(cell=decoder_cell, helper=helper, initial_state=decoder_cell.zero_state(
                    dtype=tf.float32, batch_size=batch_size))

                # The first returning argument of dynamic_decode contains two fields:
                #   rnn_output (predicted logits)
                #   sample_id (predictions)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, maximum_iterations=tf.reduce_max(self.ground_truth_lengths), 
                                                                  output_time_major=False, impute_finished=True)

                return outputs

        self.train_outputs = decode(train_helper, 'decode')
        self.infer_outputs = decode(infer_helper, 'decode', reuse=True)
        self.train_predictions = self.train_outputs.sample_id
        self.infer_predictions = self.infer_outputs.sample_id

    def define_loss_and_train(self):
        weights = tf.cast(tf.sequence_mask(self.ground_truth_lengths), dtype=tf.float32)
        self.loss = tf.contrib.seq2seq.sequence_loss(
            self.train_outputs.rnn_output,
            self.ground_truth,
            weights
        )
        self.train_op = tf.contrib.layers.optimize_loss(
            loss=self.loss,
            optimizer='Adam',
            learning_rate=self.learning_rate_ph,
            clip_gradients=1.0,
            global_step=tf.train.get_global_step()
        )

    def train_on_batch(self, session, X, X_seq_len, Y, Y_seq_len, learning_rate, dropout_keep_probability):
        feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: X_seq_len,
            self.ground_truth: Y,
            self.ground_truth_lengths: Y_seq_len,
            self.learning_rate_ph: learning_rate,
            self.dropout_ph: dropout_keep_probability
        }
        loss, _ = session.run([
            self.loss,
            self.train_op], feed_dict=feed_dict)
        return loss
    
    def get_reply(self, session, input_sentence):
        input_sentence = text_prepare(input_sentence)
        X = [[word_to_idx[word] if word in word_to_idx else start_idx for word in input_sentence]]
        X = np.array(X)
        feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: np.array([len(input_sentence)]),
            self.ground_truth_lengths: np.array([15])
        }
        pred = session.run([self.infer_predictions], feed_dict=feed_dict)
        return " ".join([idx_to_word[index] for index in pred[0][0][:-1]])

    def train(self, session, epochs, batch_size, input_sentences_idx, ground_truth_sentences_idx, input_lengths, ground_truth_lengths, learning_rate, dropout_keep_probability):
        for i in range(epochs):
            batch_num = 1
            for (batch_input_sentences_idx,
                 batch_input_length,
                 batch_ground_truth_sentences_idx,
                 batch_ground_truth_length) in batch_generator(
                batch_size, input_sentences_idx, ground_truth_sentences_idx,
                input_lengths, ground_truth_lengths):
                loss = self.train_on_batch(
                    session,
                    batch_input_sentences_idx,
                    batch_input_length,
                    batch_ground_truth_sentences_idx,
                    batch_ground_truth_length,
                    learning_rate,
                    dropout_keep_probability
                )
                print("Epoch {i}, batch {batch}, loss = {loss}".format(i=i+1, batch=batch_num, loss=loss))
                batch_num += 1

In [67]:
tf.reset_default_graph()

In [68]:
chatbot = ChatBot()

In [69]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [93]:
chatbot.train(session,
              2, 256,
              input_sentences_idx,
              ground_truth_sentences_idx,
              input_lengths,
              ground_truth_lengths,
              3e-5,
              0.5)

Epoch 1, batch 1, loss = 4.989370822906494
Epoch 1, batch 2, loss = 5.199731826782227
Epoch 1, batch 3, loss = 5.3325066566467285
Epoch 1, batch 4, loss = 5.551334381103516
Epoch 1, batch 5, loss = 5.278797626495361
Epoch 1, batch 6, loss = 4.951613426208496
Epoch 1, batch 7, loss = 5.352148056030273
Epoch 1, batch 8, loss = 5.071322441101074
Epoch 1, batch 9, loss = 5.054070472717285
Epoch 1, batch 10, loss = 4.929233551025391
Epoch 1, batch 11, loss = 5.217537879943848
Epoch 1, batch 12, loss = 5.460282802581787
Epoch 1, batch 13, loss = 5.179758548736572
Epoch 1, batch 14, loss = 5.081521987915039
Epoch 1, batch 15, loss = 4.866884231567383
Epoch 1, batch 16, loss = 5.13714075088501
Epoch 1, batch 17, loss = 4.953582763671875
Epoch 1, batch 18, loss = 4.696691036224365
Epoch 1, batch 19, loss = 4.883766174316406
Epoch 1, batch 20, loss = 4.88119649887085
Epoch 1, batch 21, loss = 4.938894271850586
Epoch 1, batch 22, loss = 5.012141227722168
Epoch 1, batch 23, loss = 5.12285804748535

Epoch 1, batch 186, loss = 5.247554302215576
Epoch 1, batch 187, loss = 5.122101783752441
Epoch 1, batch 188, loss = 4.902321815490723
Epoch 1, batch 189, loss = 4.837375640869141
Epoch 1, batch 190, loss = 4.730188369750977
Epoch 1, batch 191, loss = 4.996434211730957
Epoch 1, batch 192, loss = 4.938672065734863
Epoch 1, batch 193, loss = 4.94050407409668
Epoch 1, batch 194, loss = 4.908953666687012
Epoch 1, batch 195, loss = 5.023035526275635
Epoch 1, batch 196, loss = 5.020195007324219
Epoch 1, batch 197, loss = 4.949326515197754
Epoch 1, batch 198, loss = 5.151440143585205
Epoch 1, batch 199, loss = 4.9338297843933105
Epoch 1, batch 200, loss = 4.577725410461426
Epoch 1, batch 201, loss = 4.757533073425293
Epoch 1, batch 202, loss = 4.9821295738220215
Epoch 1, batch 203, loss = 4.77232027053833
Epoch 1, batch 204, loss = 4.8319807052612305
Epoch 1, batch 205, loss = 4.704316139221191
Epoch 1, batch 206, loss = 4.652942180633545
Epoch 1, batch 207, loss = 4.569149971008301
Epoch 1, 

Epoch 1, batch 368, loss = 4.969264030456543
Epoch 1, batch 369, loss = 4.905205726623535
Epoch 1, batch 370, loss = 4.7759833335876465
Epoch 1, batch 371, loss = 4.9342827796936035
Epoch 1, batch 372, loss = 4.845490455627441
Epoch 1, batch 373, loss = 4.702303886413574
Epoch 1, batch 374, loss = 5.051535606384277
Epoch 1, batch 375, loss = 4.632069110870361
Epoch 1, batch 376, loss = 4.750974178314209
Epoch 1, batch 377, loss = 4.958523273468018
Epoch 1, batch 378, loss = 4.648577690124512
Epoch 1, batch 379, loss = 4.718159198760986
Epoch 1, batch 380, loss = 4.863255500793457
Epoch 1, batch 381, loss = 4.838286876678467
Epoch 1, batch 382, loss = 4.726115703582764
Epoch 1, batch 383, loss = 4.663963794708252
Epoch 1, batch 384, loss = 4.721309661865234
Epoch 1, batch 385, loss = 4.581992149353027
Epoch 1, batch 386, loss = 4.624367713928223
Epoch 1, batch 387, loss = 4.535958290100098
Epoch 1, batch 388, loss = 4.725142002105713
Epoch 1, batch 389, loss = 4.76891565322876
Epoch 1, 

Epoch 1, batch 551, loss = 4.751232147216797
Epoch 1, batch 552, loss = 4.682823181152344
Epoch 1, batch 553, loss = 4.737113952636719
Epoch 1, batch 554, loss = 4.715873718261719
Epoch 1, batch 555, loss = 4.696431636810303
Epoch 1, batch 556, loss = 4.734005451202393
Epoch 1, batch 557, loss = 4.721468448638916
Epoch 1, batch 558, loss = 4.756258964538574
Epoch 1, batch 559, loss = 4.709580898284912
Epoch 1, batch 560, loss = 4.94669771194458
Epoch 1, batch 561, loss = 4.808894634246826
Epoch 1, batch 562, loss = 4.7358880043029785
Epoch 1, batch 563, loss = 4.934584140777588
Epoch 1, batch 564, loss = 4.783270359039307
Epoch 1, batch 565, loss = 4.698782444000244
Epoch 1, batch 566, loss = 4.9135589599609375
Epoch 1, batch 567, loss = 4.849909782409668
Epoch 1, batch 568, loss = 4.920575141906738
Epoch 1, batch 569, loss = 4.906631946563721
Epoch 1, batch 570, loss = 4.702907562255859
Epoch 1, batch 571, loss = 4.829619884490967
Epoch 1, batch 572, loss = 4.7317657470703125
Epoch 1,

Epoch 1, batch 734, loss = 4.818121910095215
Epoch 1, batch 735, loss = 4.959081172943115
Epoch 1, batch 736, loss = 5.028728008270264
Epoch 1, batch 737, loss = 4.948211193084717
Epoch 1, batch 738, loss = 4.847878932952881
Epoch 1, batch 739, loss = 4.919224262237549
Epoch 1, batch 740, loss = 5.122117042541504
Epoch 1, batch 741, loss = 4.939424991607666
Epoch 1, batch 742, loss = 4.922999858856201
Epoch 1, batch 743, loss = 4.897873878479004
Epoch 1, batch 744, loss = 4.87775182723999
Epoch 1, batch 745, loss = 4.975220680236816
Epoch 1, batch 746, loss = 4.960134029388428
Epoch 1, batch 747, loss = 4.972772598266602
Epoch 1, batch 748, loss = 4.997978210449219
Epoch 1, batch 749, loss = 4.984060287475586
Epoch 1, batch 750, loss = 4.983859062194824
Epoch 1, batch 751, loss = 4.923836708068848
Epoch 1, batch 752, loss = 4.761726379394531
Epoch 1, batch 753, loss = 4.846303939819336
Epoch 1, batch 754, loss = 4.985813140869141
Epoch 1, batch 755, loss = 4.907032012939453
Epoch 1, ba

...

Epoch 2, batch 428, loss = 4.980357646942139
Epoch 2, batch 429, loss = 4.857085704803467
Epoch 2, batch 430, loss = 4.6919026374816895
Epoch 2, batch 431, loss = 4.940173625946045
Epoch 2, batch 432, loss = 4.936397075653076
Epoch 2, batch 433, loss = 4.731425762176514
Epoch 2, batch 434, loss = 4.82080602645874
Epoch 2, batch 435, loss = 4.719088077545166
Epoch 2, batch 436, loss = 4.6636810302734375
Epoch 2, batch 437, loss = 4.859326362609863
Epoch 2, batch 438, loss = 4.745948791503906
Epoch 2, batch 439, loss = 4.723484992980957
Epoch 2, batch 440, loss = 4.8755974769592285
Epoch 2, batch 441, loss = 4.647143840789795
Epoch 2, batch 442, loss = 4.8740034103393555
Epoch 2, batch 443, loss = 4.951768398284912
Epoch 2, batch 444, loss = 5.001911640167236
Epoch 2, batch 445, loss = 4.85504150390625
Epoch 2, batch 446, loss = 4.97410249710083
Epoch 2, batch 447, loss = 4.983949661254883
Epoch 2, batch 448, loss = 4.790013790130615
Epoch 2, batch 449, loss = 4.879027366638184
Epoch 2, 

Epoch 2, batch 611, loss = 4.789205551147461
Epoch 2, batch 612, loss = 4.836023330688477
Epoch 2, batch 613, loss = 4.861111640930176
Epoch 2, batch 614, loss = 4.82060432434082
Epoch 2, batch 615, loss = 5.023276329040527
Epoch 2, batch 616, loss = 4.93698263168335
Epoch 2, batch 617, loss = 4.986858367919922
Epoch 2, batch 618, loss = 5.062138080596924
Epoch 2, batch 619, loss = 4.841671943664551
Epoch 2, batch 620, loss = 4.792093276977539
Epoch 2, batch 621, loss = 4.800561904907227
Epoch 2, batch 622, loss = 4.940847396850586
Epoch 2, batch 623, loss = 4.841614246368408
Epoch 2, batch 624, loss = 4.981656074523926
Epoch 2, batch 625, loss = 5.002005577087402
Epoch 2, batch 626, loss = 4.917489051818848
Epoch 2, batch 627, loss = 4.945966720581055
Epoch 2, batch 628, loss = 4.870005130767822
Epoch 2, batch 629, loss = 4.971165180206299
Epoch 2, batch 630, loss = 4.852994918823242
Epoch 2, batch 631, loss = 4.888704299926758
Epoch 2, batch 632, loss = 5.001876354217529
Epoch 2, bat

Epoch 2, batch 794, loss = 4.995962142944336
Epoch 2, batch 795, loss = 4.923344135284424
Epoch 2, batch 796, loss = 4.925938606262207
Epoch 2, batch 797, loss = 5.041539669036865
Epoch 2, batch 798, loss = 4.8770270347595215
Epoch 2, batch 799, loss = 4.940107345581055
Epoch 2, batch 800, loss = 4.964145183563232
Epoch 2, batch 801, loss = 4.882061004638672
Epoch 2, batch 802, loss = 4.926233291625977
Epoch 2, batch 803, loss = 4.904962539672852
Epoch 2, batch 804, loss = 4.866240978240967
Epoch 2, batch 805, loss = 4.974330425262451
Epoch 2, batch 806, loss = 4.952371120452881
Epoch 2, batch 807, loss = 4.886685371398926
Epoch 2, batch 808, loss = 4.940040588378906
Epoch 2, batch 809, loss = 4.988296985626221
Epoch 2, batch 810, loss = 4.650825500488281
Epoch 2, batch 811, loss = 5.2806782722473145
Epoch 2, batch 812, loss = 5.076151371002197
Epoch 2, batch 813, loss = 4.72935676574707
Epoch 2, batch 814, loss = 4.973908424377441
Epoch 2, batch 815, loss = 5.218023300170898
Epoch 2, 

Epoch 2, batch 977, loss = 4.836925983428955
Epoch 2, batch 978, loss = 4.869237899780273
Epoch 2, batch 979, loss = 4.825521469116211
Epoch 2, batch 980, loss = 4.965408802032471
Epoch 2, batch 981, loss = 4.773140907287598
Epoch 2, batch 982, loss = 4.919412612915039
Epoch 2, batch 983, loss = 4.959787845611572
Epoch 2, batch 984, loss = 4.908205509185791
Epoch 2, batch 985, loss = 4.986608505249023
Epoch 2, batch 986, loss = 4.962819576263428
Epoch 2, batch 987, loss = 4.805509090423584
Epoch 2, batch 988, loss = 5.041299819946289
Epoch 2, batch 989, loss = 4.91310977935791
Epoch 2, batch 990, loss = 5.01199197769165
Epoch 2, batch 991, loss = 4.89693546295166
Epoch 2, batch 992, loss = 4.887451648712158
Epoch 2, batch 993, loss = 4.918759822845459
Epoch 2, batch 994, loss = 5.027114391326904
Epoch 2, batch 995, loss = 4.919490337371826
Epoch 2, batch 996, loss = 4.877673625946045
Epoch 2, batch 997, loss = 4.957156181335449
Epoch 2, batch 998, loss = 4.975280284881592
Epoch 2, batc

Epoch 2, batch 1156, loss = 4.938380241394043
Epoch 2, batch 1157, loss = 4.902214527130127
Epoch 2, batch 1158, loss = 4.882693767547607
Epoch 2, batch 1159, loss = 5.002616882324219
Epoch 2, batch 1160, loss = 4.991044998168945
Epoch 2, batch 1161, loss = 4.802206993103027
Epoch 2, batch 1162, loss = 4.928064823150635
Epoch 2, batch 1163, loss = 4.916528701782227
Epoch 2, batch 1164, loss = 4.648209095001221
Epoch 2, batch 1165, loss = 4.641732692718506
Epoch 2, batch 1166, loss = 4.516732215881348
Epoch 2, batch 1167, loss = 4.648763179779053
Epoch 2, batch 1168, loss = 4.777879238128662
Epoch 2, batch 1169, loss = 4.8534255027771
Epoch 2, batch 1170, loss = 4.933951377868652
Epoch 2, batch 1171, loss = 4.814910411834717
Epoch 2, batch 1172, loss = 4.85117769241333
Epoch 2, batch 1173, loss = 4.808319091796875
Epoch 2, batch 1174, loss = 4.833578586578369
Epoch 2, batch 1175, loss = 4.977574348449707
Epoch 2, batch 1176, loss = 4.936861515045166
Epoch 2, batch 1177, loss = 4.8307275

In [96]:
chatbot.get_reply(session, "what is your name")

'i m not'